In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 
from torch.autograd import Variable as V

import sys
sys.path.append('../')
import utils
import wiki_utils
%matplotlib inline

In [2]:
batch_size = 32
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

In [3]:
eval_batch_size = 32

In [4]:
train_loader, val_loader, test_loader, TEXT = wiki_utils.WikiTexts(batch_size, sequence_length)

In [5]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()
        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers
       
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [6]:
weight_matrix = TEXT.vocab.vectors

In [7]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = weight_matrix.size(0)
    hidden = model.init_hidden(eval_batch_size)
    for i, text in enumerate(data_loader):
        data, targets = text.text, text.target
   #     output, hidden = model(data)
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
   #     total_loss += len(data) * criterion(output_flat, targets.view(-1)).item()
        total_loss += criterion(output_flat, targets.view(-1)).item()
    return total_loss/len(data_loader) 

In [8]:
def train():
    model.train()
    total_loss = 0
    ntokens = weight_matrix.size(0)
    for batch, text in enumerate(train_loader):
        data, targets = text.text, text.target
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [9]:
ntokens = weight_matrix.size(0)
model = RNNModel('LSTM', ntokens, weight_matrix.size(1), weight_matrix.size(1), 2, 0.3)
model.encoder.weight.data.copy_(weight_matrix)
criterion = nn.CrossEntropyLoss()

In [10]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()    
        s_idx = torch.multinomial(s_weights, 1)[0]       
        x.data.fill_(s_idx)             
        s = TEXT.vocab.itos[s_idx]                    
        out.append(s)
    return ' '.join(out)

In [11]:
with torch.no_grad():
    print('sample:\n', generate(20), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(20), '\n')


sample:
 leaned principals naoko themes torpedo afrodisiac honoured khalid towards gallons full sankara donors falsetto larva reprises symbols knott insurance tiered 

| epoch   1 |   100/ 2330 batches | lr 4.00 | loss  7.93 | ppl  2775.82
| epoch   1 |   200/ 2330 batches | lr 4.00 | loss  7.10 | ppl  1206.98
| epoch   1 |   300/ 2330 batches | lr 4.00 | loss  6.88 | ppl   970.26
| epoch   1 |   400/ 2330 batches | lr 4.00 | loss  6.72 | ppl   825.69
| epoch   1 |   500/ 2330 batches | lr 4.00 | loss  6.62 | ppl   748.54
| epoch   1 |   600/ 2330 batches | lr 4.00 | loss  6.52 | ppl   679.95
| epoch   1 |   700/ 2330 batches | lr 4.00 | loss  6.49 | ppl   655.71
| epoch   1 |   800/ 2330 batches | lr 4.00 | loss  6.53 | ppl   688.09
| epoch   1 |   900/ 2330 batches | lr 4.00 | loss  6.41 | ppl   606.34
| epoch   1 |  1000/ 2330 batches | lr 4.00 | loss  6.39 | ppl   593.62
| epoch   1 |  1100/ 2330 batches | lr 4.00 | loss  6.42 | ppl   616.90
| epoch   1 |  1200/ 2330 batches | lr 4

| epoch   5 |   100/ 2330 batches | lr 4.00 | loss  5.76 | ppl   317.74
| epoch   5 |   200/ 2330 batches | lr 4.00 | loss  5.65 | ppl   283.25
| epoch   5 |   300/ 2330 batches | lr 4.00 | loss  5.65 | ppl   283.72
| epoch   5 |   400/ 2330 batches | lr 4.00 | loss  5.69 | ppl   295.19
| epoch   5 |   500/ 2330 batches | lr 4.00 | loss  5.65 | ppl   283.48
| epoch   5 |   600/ 2330 batches | lr 4.00 | loss  5.63 | ppl   277.46
| epoch   5 |   700/ 2330 batches | lr 4.00 | loss  5.62 | ppl   274.88
| epoch   5 |   800/ 2330 batches | lr 4.00 | loss  5.69 | ppl   295.79
| epoch   5 |   900/ 2330 batches | lr 4.00 | loss  5.64 | ppl   280.67
| epoch   5 |  1000/ 2330 batches | lr 4.00 | loss  5.63 | ppl   279.62
| epoch   5 |  1100/ 2330 batches | lr 4.00 | loss  5.69 | ppl   294.93
| epoch   5 |  1200/ 2330 batches | lr 4.00 | loss  5.68 | ppl   291.58
| epoch   5 |  1300/ 2330 batches | lr 4.00 | loss  5.64 | ppl   281.73
| epoch   5 |  1400/ 2330 batches | lr 4.00 | loss  5.64 | ppl  

In [11]:
t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w') as outf:
    outf.write(t075)
with open('./generated1.txt', 'w') as outf:
    outf.write(t1)
with open('./generated15.txt', 'w') as outf:
    outf.write(t15)